红黑树真是又臭又长，从 STL 看到红黑树到今天基本理解与实现花了快三天时间，相当于写 vector 与 list 的时间总和，令人感叹。

## 红黑树性质

1. 节点是红色或黑色。
2. 根是黑色。（根据OI_WIKI，这一条不一定需要满足）
3. 所有叶子都是黑色（叶子是NIL节点）。
4. 每个红色节点必须有两个黑色的子节点。（从每个叶子到根的所有路径上不能有两个连续的红色节点。）
5. 从任一节点到其每个叶子的所有简单路径都包含相同数目的黑色节点。

## 红黑树结构

### 节点结构

In [58]:
class Node():
    """0 is black, 1 is red"""
    def __init__(self, val):
        self.val = val
        self.parent = None
        self.left = None
        self.right = None
        self.color = 1

### 红黑树结构

需要注意的是这里的 `TNULL` 设计，代表了红黑树中的叶子节点（空节点），但是与 `None` 不同的是，其具有 `color` 与 `parent` 等属性，这样设计可以使后续的许多判断不用单独考虑 `None` 的情况。

In [59]:
class RedBlackTree():
    def __init__(self):
        self.TNULL = Node(0)
        self.TNULL.color = 0
        self.TNULL.left = None
        self.TNULL.right = None
        self.root = self.TNULL

## 左旋与右旋

In [61]:
def left_rotate(self, node):
    '''
    左旋操作，分为三步：
    1. 将 r.parent 置为 p（可能会更新 root），根据 node 的位置把 r 连接至 p.left 或 p.right
    2. 将 node.parent 置为 r，r.left 置为 node
    3. 将 node.right 置为 rl，若 rl 存在，将 rl.parent 置为 node，
    * 左旋示意图：对节点x进行左旋
    *     p                       p
    *    /                       /
    *   node                    r
    *  / \                     / \
    * x   r     ----->      node  rr
    *    / \                /  \
    *   rl  rr             x   rl
    '''

    p = node.parent 
    r = node.right
    rl = r.left

    # 1 r 连到父节点，若父节点为空（node 为 root），更新 root
    r.parent = p
    if not p: self.root = r 
    else:
        # 1.1 根据 node 相对于 p 的位置来连接 p 与 r
        if (p.left == node): p.left = r 
        else: p.right = r 

    # 2. 连接 node 与 r
    node.parent = r
    r.left = node 
    
    # 3. 连接 rl 与 node
    node.right = rl 
    if (rl): rl.parent = node 

def right_rotate(self, node):
    ''' 
    右旋操作，分为三步：
    1. 将 l.parent 置为 p（可能会更新 root），根据 node 的位置把 l 连接至 p.left 或 p.right
    2. 将 node.parent 置为 l，l.right 置为 node
    3. 将 node.left 置为 lr，若 lr 存在，将 lr.parent 置为 node，
    * 右旋示意图：对节点y进行右旋
    *         p                   p
    *        /                   /
    *      node                 l
    *     /    \               / \
    *    l      r   ----->   ll  node
    *   / \                      /  \
    * ll   lr                   lr   r
    '''
    p = node.parent
    l = node.left
    lr = l.right

    l.parent = p
    if not p: self.root = l
    else:
        if (p.left == node): p.left = l 
        else: p.right = l 

    node.parent = l 
    l.right = node 

    node.left = lr
    if (lr): lr.parent = node 

## 节点插入及重新平衡

### 节点插入

In [62]:
def insert_node(self, key):
    node = Node(key)
    node.parent = None
    node.left = self.TNULL  # 将左右节点都设置为 TNULL，即叶子节点
    node.right = self.TNULL
    node.color = 1          # 新插入的节点为红色

    y = None
    x = self.root

    # 从根节点开始，找到新插入节点的位置
    while x != self.TNULL:
        y = x
        if node.val < x.val: x = x.left
        else: x = x.right

    # 插入新节点
    node.parent = y
    if y == None: self.root = node
    elif node.val < y.val: y.left = node
    else: y.right = node

    self.insert_fix(node)  # 修正红黑树

### 插入后修正

在讨论插入情况之前，先做如下表述上的约定：新插入节点为 Node，其父节点为 P， 祖父节点为 G，叔叔节点为 U，插入节点与父节点的相对位置为 L/R。

新插入节点初始化为红色节点，插入节点时有以下几种情况：

1. 插入节点为根节点，直接将节点颜色置为黑色
2. 插入节点的父节点为黑色，不需要做任何操作
3. 插入节点的父节点为红色，需要进行调整
    1. 插入节点的叔叔节点为红色，将父节点和叔叔节点置为黑色，祖父节点置为红色，将祖父节点作为新的插入节点，继续调整

        <a href="https://sm.ms/image/oe9up81XIDctEiW" target="_blank"><img src="https://s2.loli.net/2023/08/16/oe9up81XIDctEiW.png" ></a>

    2. 插入节点的叔叔节点为黑色，且插入节点与父节点同侧（LL/RR），以祖父节点为支点进行反向旋转（R/L），将父节点置为黑色，祖父节点置为红色。

        <a href="https://sm.ms/image/ycVxGpQLKWfeXa1" target="_blank"><img src="https://s2.loli.net/2023/08/16/ycVxGpQLKWfeXa1.png" ></a>

    3. 插入节点的叔叔节点为黑色，且插入节点与父节点异侧（LR/RL），以父节点为支点进行反向旋转（L/R），转化为 3.2。

        <a href="https://sm.ms/image/P8jkMXG7gLsEHi6" target="_blank"><img src="https://s2.loli.net/2023/08/16/P8jkMXG7gLsEHi6.png" ></a>

可以看出，由于将新节点都设置为了红色节点，因此插入操作有可能违背的就只有性质4，即红色节点的父节点不能为红色。

In [64]:
def insert_fix(self, node):
    # 目前正在处理的节点
    cur = node  
    # 父节点为黑色不用处理，case2
    while (cur.parent and cur.parent.color == 1):  
        p = cur.parent
        g = cur.parent.parent
        # L
        if (p == g.left): 
            u = g.right
            # uncle 红色，case3.1
            # 由于设置了 TNULL，因此不用判断 u 是否为 None
            if (u.color == 1):    
                p.color = u.color = 0
                g.color = 1
                # 更新当前处理节点切换为祖父节点
                cur = g  
            else:
                # LR, case3.3 转化为 case3.2
                if (cur == p.right): self.left_rotate(p)
                p.color = 0
                g.color = 1
                self.right_rotate(g)
        # R
        else:
            u = g.left
            if (u.color == 1):
                p.color = u.color = 0
                g.color = 1
                cur = g
            else:
                # RL, 转为 case3.2
                if (cur == p.left): self.right_rotate(p)
                self.left_rotate(g)
                p.color = 0
                g.color = 1
        
        if (cur == self.root): break
    # 根节点始终为黑色
    self.root.color = 0

## 节点删除及树修复

红黑树的节点删除操作逻辑上分为两部分，节点的删除以及树的重新平衡，实际上在许多代码中也将删除操作分为两个函数来完成，这里的实现也按照此逻辑。

笔者认为，网络上包括书籍中的很多关于红黑树删除的教程，之所以让人看了云里雾里，是因为它们在演示中并没有将删除以及重新平衡这两个步骤分开来展示，在列举各种情况时总是把删除以及重新平衡的情况放一起说完，这样就会导致逻辑的混乱，且不利于记忆写代码的步骤。

### 节点删除

就节点删除这一个步骤来看，红黑树的节点删除操作与BST（二叉搜索树）基本相同，可分为三种情况：


1. 删除只有一个子树的节点：删除原节点，用单独的子节点代替被删除节点
2. 删除没有子树的节点：直接删除原节点，相当于用一个空节点代替了被删除节点
3. 删除有两个子树的节点：寻找后继或前继节点（这里选择后继），将待删除节点的值变为后继节点的值，再删除后继节点，删除后继节点一定会变成 1 或 2 的情况（可以思考一下为什么）

但由于红黑树的特殊性（多了一个颜色属性），因此要记录被删除节点的颜色，以应用到后续的平衡步骤中。

In [65]:
def findMin(self, node):
    """
    找到以 node 节点为根节点的树的最小值节点 并返回
    """
    temp_node = node
    # 判断条件也要为 TNULL
    while temp_node.left != self.TNULL: temp_node = temp_node.left
    return temp_node

def transplant(self, node1, node2):
    """
    辅助函数，用 node2 替换 node1
    """
    p1 = node1.parent
    if not p1: self.root = node2
    elif (node1 == p1.left): p1.left = node2
    elif (node1 == p1.right): p1.right = node2
    # 由于 node2 必定为 TNULL，不会为 None，因此一定有 parent
    node2.parent = p1  

> `transplant` 这个函数很多博客解释的都简单地解释为用 node2 替换 node1，但实际上并非如此。这里只考虑了 node1 的parent 指针与 node2 的交接而没有考虑两个节点的 left 及 right 指针的问题，因为在大部分操作中，node2 为 node1 唯一的子节点，而 node1 即将被删除，不需要考虑 left 及 right 指针，需要维护 left 及 right 指针时会在执行完函数后单独处理。

In [44]:
def delete_node(self, node, key):
    
    z = self.TNULL
    # 找到 key 对应的节点
    while node != self.TNULL:
        if node.val == key: z = node
        if node.val <= key: node = node.right
        else: node = node.left
    # 如果没有找到 key 对应的节点，返回
    if z == self.TNULL:
        print("Cannot find key in the tree")
        return
    
    node = z
    node_color = node.color
    temp_node = None  # 替换到被删除节点（node）处的节点

    # ===================== case1 与 case2 ================== //
    # node 本身被删除
    if not node.left:
        temp_node = node.right
        self.transplant(node, node.right)
    elif not node.right:
        temp_node = node.left
        is_temp_node_left = True
        self.transplant(node, node.left)
    # ========================= case 3 ====================== //
    # node 的后继被删除
    else:
        # 找到后继节点，及右子树中的最小节点，该节点将代替 node 被删除
        successor = self.findMin(node.right)
        node_color = successor.color
        # temp_node 将在 successor 被删除后代替它
        temp_node = successor.right
        # 将 node 替换为 successor，相当于仅替换值
        node.val = successor.val
        # 删除 successor，用 temp_node 替代
        self.transplant(successor, temp_node)
        
    # 这里的 delete_rebalance 为红黑树的重新平衡操作，需要注意的是，
    # 重新平衡的操作总是在被删除节点的位置进行的，即上面维护的 temp_node 最终的位置
    # 仅在被删除节点的颜色（node_color 维护）为黑色时才需重新平衡树
    if (node_color == 0): self.delete_rebalance(temp_node)

### 红黑树修复

在讨论红黑树的重新平衡之前，回顾一下在红黑树的节点删除部分我们做了什么：首先，根据将要被删除的节点的孩子数量，来决定删除策略，其中零或一个孩子的情况较好处理，而两个孩子的情况则可以通过寻找后继节点的方式转换为前两种情况，需要注意的是，前两种情况删除的 node 位置即为最初的 node 位置，而后一种情况删除的节点位置已经不在最初的 node 的位置了，而是其后继所在的位置。

在删除完节点后，temp_node 所指向的节点为 **替代被删除节点的节点**，它的位置在被删除节点的位置，后续的所有的修复情况均以 temp_node 的位置及其相关节点的颜色作为判断标准。

其次，不难注意到，只有当被删除节点颜色为黑色时，才会增加重新平衡红黑树的操作，否则红黑树不需要调整，因此。进入 `delete_rebalance` 函数时，红黑树的情况为：**所有通过 `temp_node` 的路径均少了一个黑色节点**（因为原本 `temp_node` 所在位置为一个黑色节点，现在被删了）。因此，`delete_rebalance` 的目标（作用）即为：**为通过 `temp_node` 节点的路径增加一个黑色节点，同时保证其他路径的黑色节点数目不变**，才能重新平衡红黑树。

> 注意，这里的新增指的是通过将红色节点变为黑色节点的方式来增加黑色节点数目，而不是插入新的节点。

弄清楚了这些，才能使我们更加容易理解红黑树修复的各种情况。

首先进行一些定义，这里以 N 代表 temp_node，S 代表 N 的兄弟节点，P 代表 N 与 S 的父节点，SL 与 SR 分别代表 S 的左右节点，节点 S 的同侧儿子表示的情况为：若 S 为 P 的左儿子，则 S 的左儿子为 S 的同侧儿子（SL），反之 SR 为同侧儿子，异侧儿子定义类似。

红黑树修复有以下六种情况：

1. N 为新的根节点

    这说明原始被删除节点即为根节点，这种情况相当于所有路径均少了一个黑色的节点，我们将 N 变为黑色，即完成操作。

2. S 为红色
    
    <a href="https://sm.ms/image/BCYc1jXDpwSbGqT" target="_blank"><img src="https://s2.loli.net/2023/08/16/BCYc1jXDpwSbGqT.png" alt="3.png"></a>
    
    这种情况下无法直接处理，需要在 P 上做旋转，转换为其他情况。

3. N，S，P 均为黑色
    
    <a href="https://sm.ms/image/om5ueIjxwrq7tfs" target="_blank"><img src="https://s2.loli.net/2023/08/16/om5ueIjxwrq7tfs.png" ></a>
    
    这种情况下无法在这个局部结构中新增一个黑色节点（牢记我们的目标，在 N 的路径上增加一个黑色节点），此时只能将 S 变为红色，这样会使得局部结构达到平衡（**左右都少了一个黑色节点**），但红黑树整体依然不平衡。需要继续向上调整，试图在更上层的结构中为这条路径增加一个黑色节点。
    
    因此，以 P 作为 `delete_rebalance` 的参数，改为处理 P 节点。至于为什么要处理该节点，牢记：**`delete_rebalance` 的作用为使过 `temp_node` 节点的路径增加一个黑色节点，同时保证其他路径的黑色节点数目不变**，在上述的调整过程中，我们通过改变 S 的颜色的方式使这个局部结构达到了平衡，但相较于其他路径都少了一个黑色节点，因此需要处理 P 节点，使得通过 P 节点的路径增加一个黑色节点，以平衡红黑树。

4. S、SL、SR 均为黑色，P 为红色

    <a href="https://sm.ms/image/z1RtU4wWOxpcs8Z" target="_blank"><img src="https://s2.loli.net/2023/08/16/z1RtU4wWOxpcs8Z.png" ></a>

    这种情况比较好处理，简单地将 S 变为红色再将 P 变为黑色，就能使得 N 的路径上多出一个黑色节点，而其他路径黑色节点数量不变。

5. S 为黑色，S 的同侧儿子为红色
    
    <a href="https://sm.ms/image/CbcfexgIvXHWqN1" target="_blank"><img src="https://s2.loli.net/2023/08/16/CbcfexgIvXHWqN1.png" ></a>

    这种情况下我们旋转 P，交换 S 与 P 的颜色，并且将 S 的同侧儿子颜色变为黑色，就能使得所有通过 N 的路径均增加一个黑色节点而其他路径黑色节点数量不变。

6. S 为黑色，S 的同侧儿子为黑色，异侧儿子为红色
    
    <a href="https://sm.ms/image/EV3tUwbDsYBF6l1" target="_blank"><img src="https://s2.loli.net/2023/08/16/EV3tUwbDsYBF6l1.png" ></a>

    这种情况也不能直接处理，需要通过旋转 S 再交换 S 与它的新父亲颜色的方式转为 case5 的情况再进一步处理。



In [45]:
def delete_rebalance(self, node):
    while (node != self.root and node.color == 0):
        if (node == node.parent.left):
            s = node.parent.right
            # case2：兄弟节点为红色节点，旋转父节点
            if (s.color == 1):
                node.parent.color = 1
                s.color = 0
                self.left_rotate(node.parent)
                s = node.parent.right  # 更新兄弟节点
            # case3: 兄弟节点的两个子节点均为黑色，父节点也为黑色？
            if (not s.left or s.left.color == 0) and (not s.right or s.right.color == 0):
                s.color = 1
                node = node.parent  # 向上继续处理
            else:
                # case5: 兄弟节点同侧的节点为黑色
                if (not s.right or s.right.color == 0):
                    s.color = 1
                    s.left.color = 0
                    self.right_rotate(s)
                    s = node.parent.right  # 更新兄弟节点
                # 处理之后变为 case6
                # case6: 兄弟节点的同侧节点为红色
                s.color = node.parent.color
                node.parent.color = 0
                s.right.color = 0
                self.left_rotate(node.parent)
            node = self.root 
            break
        # 对称操作
        else:
            s = node.parent.left
            if (s.color == 1):
                node.parent.color = 1
                s.color = 0
                self.right_rotate(node.parent)
                s = node.parent.left
            if (not s.left or s.left.color == 0) and (not s.right or s.right.color == 0):
                s.color = 1
                node = node.parent
            else:
                if (not s.left or s.left.color == 0):
                    s.color = 1
                    s.right.color = 0
                    self.left_rotate(s)
                    s = node.parent.left
                s.color = node.parent.color
                node.parent.color = 0
                s.left.color = 0
                self.right_rotate(node.parent)
            node = self.root
            break
        node.color = 0

## 整体测试

In [11]:
class RB_Node:
    '''0 is black, 1 is red'''
    def __init__(self, val, color = 1):
        self.val = val
        self.color = color
        self.left = None
        self.right = None
        self.parent = None
    
    def is_black_node(self):
        return self.color == 0

    def set_black_node(self):
        self.color = 0
    
    def set_red_node(self):
        self.color = 1

    def print(self):
        '''中序遍历打印树'''
        if self.left: self.left.print()
        # print((self.val, self.color))
        print(self.val)
        if self.right: self.right.print()

class RB_Tree:
    def __init__(self, root=None):
        self.root = root
    
    def print(self):
        self.root.print()

    def findMin(self, node):
        """
        找到以 node 节点为根节点的树的最小值节点 并返回
        """
        temp_node = node
        while temp_node.left: temp_node = temp_node.left
        return temp_node

    def transplant(self, node1, node2):
        """
        辅助函数，用 node2 替换 node1
        """
        p1 = node1.parent
        if not p1: self.root = node2
        elif (node1 == p1.left): p1.left = node2
        elif (node1 == p1.right): p1.right = node2
        if node2: node2.parent = p1

    def left_rotate(self, node: RB_Node):
        p = node.parent 
        r = node.right
        rl = r.left

        # 1 r 连到父节点，若父节点为空（node 为 root），更新 root
        r.parent = p
        if not p: self.root = r 
        else:
            # 1.1 根据 node 相对于 p 的位置来连接 p 与 r
            if (p.left == node): p.left = r 
            else: p.right = r 

        # 2. 连接 node 与 r
        node.parent = r
        r.left = node 
        
        # 3. 连接 rl 与 node
        node.right = rl 
        if (rl): rl.parent = node 

    def right_rotate(self, node: RB_Node):
        p = node.parent
        l = node.left
        lr = l.right

        l.parent = p
        if not p: self.root = l
        else:
            if (p.left == node): p.left = l 
            else: p.right = l 

        node.parent = l 
        l.right = node 

        node.left = lr
        if (lr): lr.parent = node 

    def insert_node(self, node: RB_Node):
        # 找到最接近的节点
        temp_root = self.root
        temp_node = None
        while temp_root:
            temp_node = temp_root
            if (node.val == temp_node.val): return False
            elif (node.val > temp_node.val): temp_root = temp_root.right
            else: temp_root = temp_root.left
        # 在相应位置插入节点
        if not temp_node:
            # insert_case1
            self.root = node
            node.color = 0
        elif (node.val < temp_node.val):
            temp_node.left = node
            node.parent = temp_node
        else:
            temp_node.right = node
            node.parent = temp_node
        # 调整树
        self.insert_rebalance(node)
    
    def insert_rebalance(self, node):
        # 目前正在处理的节点
        cur = node  
        # 父节点为黑色不用处理，2
        while (cur.parent and cur.parent.color == 1):  
            p = cur.parent
            g = cur.parent.parent
            # L
            if (p == g.left): 
                u = g.right
                # uncle 红色，3.1
                if (u and u.color == 1):  
                    p.color = u.color = 0
                    g.color = 1
                    # 更新当前处理节点切换为祖父节点
                    cur = g  
                else:
                    # LR, 3.3 转化为 3.2
                    if (cur == p.right): self.left_rotate(p)
                    p.color = 0
                    g.color = 1
                    self.right_rotate(g)
            # R
            else:
                u = g.left
                if (u and u.color == 1):
                    p.color = u.color = 0
                    g.color = 1
                    cur = g
                else:
                    # RL, 转为 3.2
                    if (cur == p.left): self.right_rotate(p)
                    self.left_rotate(g)
                    p.color = 0
                    g.color = 1
            
            if (cur == self.root): break
        # 根节点始终为黑色
        self.root.color = 0
    
    def delete_node(self, node):
        node_color = node.color
        temp_node = None  # 替换到被删除节点（node）处的节点

        # ===================== case1 与 case2 ================== //
        # node 本身被删除
        if not node.left:
            temp_node = node.right
            self.transplant(node, node.right)
        elif not node.right:
            temp_node = node.left
            self.transplant(node, node.left)
        
        temp_node_parent = None if not node.parent else node.parent
        is_temp_node_left = True if (temp_node_parent and temp_node_parent.left == node) else False

        # ========================= case 3 ====================== //
        # node 的后继被删除
        if (node.left and node.right):
            # 找到后继节点，即右子树中的最小节点，该节点将代替 node 被删除
            successor = self.findMin(node.right)
            node_color = successor.color
            
            temp_node_parent = successor.parent
            is_temp_node_left = True if (temp_node_parent and temp_node_parent.left == successor) else False
            
            # temp_node 将在 successor 被删除后代替它
            temp_node = successor.right
            # 将 node 替换为 successor，相当于仅替换值
            node.val = successor.val
            # 删除 successor，用 temp_node 替代
            self.transplant(successor, temp_node)
        
        printTree(self.root)

        # 这里的 delete_rebalance 为红黑树的重新平衡操作，需要注意的是，
        # 重新平衡的操作总是在被删除节点的位置进行的，即上面维护的 temp_node 最终的位置
        # 仅在被删除节点的颜色（node_color 维护）为黑色时才需重新平衡树
        if (node_color == 0): self.delete_rebalance(temp_node, temp_node_parent, is_temp_node_left)

    def delete_rebalance(self, node, node_parent, is_node_left):
        if (not node):
            flag = True
            node = RB_Node(None, 0)
            node.parent = node_parent
            if node_parent:
                if (is_node_left): node_parent.left = node
                else: node_parent.right = node

        printTree(self.root)

        while (node != self.root and node.color == 0):
            if (node == node.parent.left):
                s = node.parent.right
                # case2：兄弟节点为红色节点，旋转父节点
                if (s.color == 1):
                    node.parent.color = 1
                    s.color = 0
                    self.left_rotate(node.parent)
                    s = node.parent.right  # 更新兄弟节点
                # case3: 兄弟节点的两个子节点均为黑色
                if (not s.left or s.left.color == 0) and (not s.right or s.right.color == 0):
                    s.color = 1
                    node = node.parent  # 向上继续处理
                else:
                    # case5: 兄弟节点同侧的节点为黑色
                    if (not s.right or s.right.color == 0):
                        s.color = 1
                        s.left.color = 0
                        self.right_rotate(s)
                        s = node.parent.right  # 更新兄弟节点
                    # 处理之后变为 case6
                    # case6: 兄弟节点的同侧节点为红色
                    s.color = node.parent.color
                    node.parent.color = 0
                    s.right.color = 0
                    self.left_rotate(node.parent)
                # node = self.root 
                break
            # 对称操作
            else:
                s = node.parent.left
                if (s.color == 1):
                    node.parent.color = 1
                    s.color = 0
                    self.right_rotate(node.parent)
                    s = node.parent.left
                if (not s.left or s.left.color == 0) and (not s.right or s.right.color == 0):
                    s.color = 1
                    node = node.parent
                else:
                    if (not s.left or s.left.color == 0):
                        s.color = 1
                        s.right.color = 0
                        self.left_rotate(s)
                        s = node.parent.left
                    s.color = node.parent.color
                    node.parent.color = 0
                    s.left.color = 0
                    self.right_rotate(node.parent)
                # node = self.root
                break
        if (flag): self.transplant(node, None)
        self.root.color = 0

In [4]:
def printTree(root, color = True):
    print("******************************************************")
    
    def maxDepth(root):
        if not root: return -1
        leftDepth = maxDepth(root.left)
        rightDepth = maxDepth(root.right)
        return max(leftDepth, rightDepth) + 1
    
    h = maxDepth(root)
    m = h + 1
    n = pow(2, m) - 1
    res = [["" for i in range(n)] for i in range(m)]
    nodes = [(root, 0, (n-1)//2)]
    
    while nodes:
        cur, r, c = nodes.pop()
        res[r][c] = str(cur.val)
        if color: res[r][c] += "r" if cur.color == 1 else "b"
        if cur.left: nodes.append((cur.left, r+1, c-pow(2,h-r-1)))
        if cur.right: nodes.append((cur.right, r+1, c+pow(2,h-r-1)))
    
    return res

In [13]:
t = RB_Tree()
t.insert_node(RB_Node(1))
t.insert_node(RB_Node(2))
t.insert_node(RB_Node(3))
t.insert_node(RB_Node(4))
t.insert_node(RB_Node(5))
t.insert_node(RB_Node(6))
t.insert_node(RB_Node(7))
t.insert_node(RB_Node(8))

In [14]:
printTree(t.root)

******************************************************


[['', '', '', '', '', '', '', '4b', '', '', '', '', '', '', ''],
 ['', '', '', '2r', '', '', '', '', '', '', '', '6r', '', '', ''],
 ['', '1b', '', '', '', '3b', '', '', '', '5b', '', '', '', '7b', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '8r']]

In [15]:
t.root.right.right.right.val 

8

In [16]:
t.delete_node(t.root)

******************************************************
******************************************************


In [7]:
printTree(t.root)

[['', '', '', '5b', '', '', ''],
 ['', '2r', '', '', '', '7r', ''],
 ['1b', '', '3b', '', '6b', '', '8b']]

In [8]:
t.delete_node(t.root)
# printTree(t.root)

In [9]:
printTree(t.root)

[['', '', '', '6b', '', '', ''],
 ['', '2r', '', '', '', '', ''],
 ['1b', '', '3b', '', '', '', '']]

In [51]:
import sys
# 创建节点
class Node():
    def __init__(self, val):
        self.val = val
        self.parent = None
        self.left = None
        self.right = None
        self.color = 1

class RedBlackTree():
    def __init__(self):
        self.TNULL = Node(0)
        self.TNULL.color = 0
        self.TNULL.left = None
        self.TNULL.right = None
        self.root = self.TNULL

    def printTree(self, color = True):
        
        def maxDepth(root):
            if root == self.TNULL: return -1
            leftDepth = maxDepth(root.left)
            rightDepth = maxDepth(root.right)
            return max(leftDepth, rightDepth) + 1
        
        h = maxDepth(self.root)
        m = h + 1
        n = pow(2, m) - 1
        res = [["" for i in range(n)] for i in range(m)]
        nodes = [(self.root, 0, (n-1)//2)]
        
        while nodes:
            cur, r, c = nodes.pop()
            res[r][c] = str(cur.val)
            if color: res[r][c] += "r" if cur.color == 1 else "b"
            if cur.left != self.TNULL: nodes.append((cur.left, r+1, c-pow(2,h-r-1)))
            if cur.right != self.TNULL: nodes.append((cur.right, r+1, c+pow(2,h-r-1)))
        
        return res

    # 前序
    def pre_order_helper(self, node):
        if node != self.TNULL:
            sys.stdout.write(node.val + " ")
            self.pre_order_helper(node.left)
            self.pre_order_helper(node.right)

    # 中序
    def in_order_helper(self, node):
        if node != self.TNULL:
            self.in_order_helper(node.left)
            sys.stdout.write(str(node.val) + " ")
            self.in_order_helper(node.right)

# 后根
    def post_order_helper(self, node):
        if node != self.TNULL:
            self.post_order_helper(node.left)
            self.post_order_helper(node.right)
            sys.stdout.write(node.val + " ")

    # 搜索树
    def search_tree_helper(self, node, key):
        if node == self.TNULL or key == node.val:
            return node

        if key < node.val:
            return self.search_tree_helper(node.left, key)
        return self.search_tree_helper(node.right, key)

    # 删除后平衡树
    def delete_fix(self, x):
        while x != self.root and x.color == 0:
            if x == x.parent.left:
                s = x.parent.right
                if s.color == 1:
                    s.color = 0
                    x.parent.color = 1
                    self.left_rotate(x.parent)
                    s = x.parent.right

                if s.left.color == 0 and s.right.color == 0:
                    s.color = 1
                    x = x.parent
                else:
                    if s.right.color == 0:
                        s.left.color = 0
                        s.color = 1
                        self.right_rotate(s)
                        s = x.parent.right

                    s.color = x.parent.color
                    x.parent.color = 0
                    s.right.color = 0
                    self.left_rotate(x.parent)
                    x = self.root
            else:
                s = x.parent.left
                if s.color == 1:
                    s.color = 0
                    x.parent.color = 1
                    self.right_rotate(x.parent)
                    s = x.parent.left

                if s.right.color == 0 and s.right.color == 0:
                    s.color = 1
                    x = x.parent
                else:
                    if s.left.color == 0:
                        s.right.color = 0
                        s.color = 1
                        self.left_rotate(s)
                        s = x.parent.left

                    s.color = x.parent.color
                    x.parent.color = 0
                    s.left.color = 0
                    self.right_rotate(x.parent)
                    x = self.root
        x.color = 0

    def __rb_transplant(self, u, v):
        if u.parent == None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    # 节点删除
    def delete_node_helper(self, node, key):
        z = self.TNULL
        while node != self.TNULL:
            if node.val == key:
                z = node

            if node.val <= key:
                node = node.right
            else:
                node = node.left

        if z == self.TNULL:
            print("Cannot find key in the tree")
            return

        y = z
        y_original_color = y.color
        if z.left == self.TNULL:
            x = z.right
            self.__rb_transplant(z, z.right)
        elif (z.right == self.TNULL):
            x = z.left
            self.__rb_transplant(z, z.left)
        else:
            y = self.minimum(z.right)
            y_original_color = y.color
            x = y.right
            # if y.parent == z:
            #     x.parent = y
            # else:
            #     self.__rb_transplant(y, y.right)
            #     y.right = z.right
            #     y.right.parent = y

            # self.__rb_transplant(z, y)
            # y.left = z.left
            # y.left.parent = y
            # y.color = z.color
            z.val = y.val
            self.__rb_transplant(y, x)

        if y_original_color == 0:
            self.delete_fix(x)

    # 插入后平衡树
    def fix_insert(self, k):
        while k.parent.color == 1:
            if k.parent == k.parent.parent.right:
                u = k.parent.parent.left
                if u.color == 1:
                    u.color = 0
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    k = k.parent.parent
                else:
                    if k == k.parent.left:
                        k = k.parent
                        self.right_rotate(k)
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    self.left_rotate(k.parent.parent)
            else:
                u = k.parent.parent.right

                if u.color == 1:
                    u.color = 0
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    k = k.parent.parent
                else:
                    if k == k.parent.right:
                        k = k.parent
                        self.left_rotate(k)
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    self.right_rotate(k.parent.parent)
            if k == self.root:
                break
        self.root.color = 0

    # Printing the tree
    def __print_helper(self, node, indent, last):
        if node != self.TNULL:
            sys.stdout.write(indent)
            if last:
                sys.stdout.write("R----")
                indent += "     "
            else:
                sys.stdout.write("L----")
                indent += "|    "

            s_color = "RED" if node.color == 1 else "BLACK"
            print(str(node.val) + "(" + s_color + ")")
            self.__print_helper(node.left, indent, False)
            self.__print_helper(node.right, indent, True)

    def preorder(self):
        self.pre_order_helper(self.root)

    def inorder(self):
        self.in_order_helper(self.root)

    def postorder(self):
        self.post_order_helper(self.root)

    def searchTree(self, k):
        return self.search_tree_helper(self.root, k)

    def minimum(self, node):
        while node.left != self.TNULL:
            node = node.left
        return node

    def maximum(self, node):
        while node.right != self.TNULL:
            node = node.right
        return node

    def successor(self, x):
        if x.right != self.TNULL:
            return self.minimum(x.right)

        y = x.parent
        while y != self.TNULL and x == y.right:
            x = y
            y = y.parent
        return y

    def predecessor(self,  x):
        if (x.left != self.TNULL):
            return self.maximum(x.left)

        y = x.parent
        while y != self.TNULL and x == y.left:
            x = y
            y = y.parent

        return y

    def left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.TNULL:
            y.left.parent = x

        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    def right_rotate(self, x):
        y = x.left
        x.left = y.right
        if y.right != self.TNULL:
            y.right.parent = x

        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y
        y.right = x
        x.parent = y

    def insert(self, key):
        node = Node(key)
        node.parent = None
        node.val = key
        node.left = self.TNULL
        node.right = self.TNULL
        node.color = 1

        y = None
        x = self.root

        while x != self.TNULL:
            y = x
            if node.val < x.val:
                x = x.left
            else:
                x = x.right

        node.parent = y
        if y == None:
            self.root = node
        elif node.val < y.val:
            y.left = node
        else:
            y.right = node

        if node.parent == None:
            node.color = 0
            return

        if node.parent.parent == None:
            return

        self.fix_insert(node)

    def get_root(self):
        return self.root

    def delete_node(self, val):
        self.delete_node_helper(self.root, val)

    def print_tree(self):
        self.__print_helper(self.root, "", True)

In [52]:
bst = RedBlackTree()

bst.insert(1)
bst.insert(2)
bst.insert(3)
bst.insert(4)
bst.insert(5)
bst.insert(6)
bst.insert(7)
bst.insert(8)
bst.insert(9)
# bst.insert(10)

# bst.print_tree()

# print("\nAfter deleting an element")
# bst.delete_node(40)
# bst.print_tree()

In [53]:
bst.printTree()

[['', '', '', '', '', '', '', '4b', '', '', '', '', '', '', ''],
 ['', '', '', '2r', '', '', '', '', '', '', '', '6r', '', '', ''],
 ['', '1b', '', '', '', '3b', '', '', '', '5b', '', '', '', '8b', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '7r', '', '9r']]

In [54]:
bst.delete_node(4)
bst.printTree()

[['', '', '', '', '', '', '', '5b', '', '', '', '', '', '', ''],
 ['', '', '', '2r', '', '', '', '', '', '', '', '8r', '', '', ''],
 ['', '1b', '', '', '', '3b', '', '', '', '6b', '', '', '', '9b', ''],
 ['', '', '', '', '', '', '', '', '', '', '7r', '', '', '', '']]

In [56]:
bst.delete_node(5)
bst.printTree()

[['', '', '', '6b', '', '', ''],
 ['', '2r', '', '', '', '8r', ''],
 ['1b', '', '3b', '', '7b', '', '9b']]

In [57]:
bst.in_order_helper(bst.root)

1 2 3 6 7 8 9 